# Predicting SPN 5246 Occurrences

## Introduction
In this notebook, we aim to create a model that predicts the occurrence of SPN 5246 faults in a trucking diagnostics dataset. SPN 5246 signifies a critical situation that can result in truck towing and costs over $4,000. The dataset is from the Big G trucking company and includes several years of onboard computer sensor readings. Our objective is to help the Big G company avoid SPN 5246 codes by using feature engineering, data exploration, and machine learning algorithms to develop an accurate predictive model.

## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
import pickle

## Data Preparation

### Load Data from Pickle File

In [2]:
# Timeseries, equiptmnet, and SPN features
df = pd.read_pickle("../data/spn_only.pkl")
df

,EquipmentID,SPN_0,SPN_16,SPN_27,SPN_33,SPN_37,SPN_38,SPN_51,SPN_70,SPN_74,...,SPN_520413,SPN_520953,SPN_521032,SPN_523530,SPN_523531,SPN_523543,SPN_524033,SPN_524037,SPN_524071,SPN_524287
EventTimeStamp,,,,,,,,,,,,,,,,,,,,,
2015-02-21 10:47:13,1439,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-02-21 11:34:34,1439,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-02-21 11:35:31,1369,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-02-21 11:35:33,1369,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-02-21 11:39:41,1674,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-06 14:00:26,2282,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03-06 14:04:23,1994,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03-06 14:13:38,1850,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Feature Selection

In [3]:
# Select features and target (SPN 5246)
X = df.drop(["SPN_5246"], axis=1)
y = df
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.25, random_state = 3)

### Apply rolling window


In [4]:
# Resample data on a daily basis
one_hot_fault_daily = df.groupby('EquipmentID').resample('D').max()

# Shift the one-hot encoded variables one day into the future
#one_hot_fault_daily_shifted = one_hot_fault_daily.groupby('EquipmentID').shift(periods=-1, fill_value=0)

# Merge the one-hot encoded variables with the other features
#df_features = pd.concat([df.drop(['SPN_5246'], axis=1), one_hot_fault_hourly_shifted], axis=1)

# Drop all rows containing NaN values
one_hot_fault_daily.dropna(inplace=True)

In [5]:
one_hot_fault_daily

EquipmentID  SPN_0  SPN_16  SPN_27  SPN_33  SPN_37  \
EquipmentID EventTimeStamp                                                      
301         2015-05-11           301.0    0.0     0.0     0.0     0.0     0.0   
            2015-05-13           301.0    0.0     0.0     0.0     0.0     0.0   
            2015-05-18           301.0    0.0     0.0     0.0     0.0     0.0   
            2015-05-21           301.0    0.0     0.0     0.0     0.0     0.0   
            2015-05-28           301.0    0.0     0.0     0.0     0.0     0.0   
...                                ...    ...     ...     ...     ...     ...   
310         2018-08-28             310    0.0     0.0     0.0     0.0     0.0   
            2018-09-06             310    0.0     0.0     0.0     0.0     0.0   
R1762       2015-02-24           R1762    0.0     0.0     0.0     0.0     0.0   
            2015-02-26           R1762    0.0     0.0     0.0     0.0     0.0   
R1764       2015-02-25           R1764    0.0     0.0     0.0     0.0     0.0   

                            SPN_38  SPN_51  SPN_70  SPN_74  ...  SPN_520413  \
EquipmentID EventTimeStamp                                  ...               
301         2015-05-11         0.0     0.0     0.0     0.0  ...         0.0   
            2015-05-13         0.0     0.0     0.0     0.0  ...         0.0   
            2015-05-18         0.0     0.0     0.0     0.0  ...         0.0   
            2015-05-21         0.0     0.0     0.0     0.0  ...         0.0   
            2015-05-28         0.0     0.0     0.0     0.0  ...         0.0   
...                            ...     ...     ...     ...  ...         ...   
310         2018-08-28         0.0     0.0     0.0     0.0  ...         0.0   
            2018-09-06         0.0     0.0     0.0     0.0  ...         0.0   
R1762       2015-02-24         0.0     0.0     0.0     0.0  ...         0.0   
            2015-02-26         0.0     0.0     0.0     0.0  ...         0.0   
R1764       2015-02-25         0.0     0.0     0.0     0.0  ...         0.0   

                            SPN_520953  SPN_521032  SPN_523530  SPN_523531  \
EquipmentID EventTimeStamp                                                   
301         2015-05-11             0.0         0.0         0.0         0.0   
            2015-05-13             0.0         0.0         0.0         0.0   
            2015-05-18             0.0         0.0         0.0         0.0   
            2015-05-21             0.0         0.0         0.0         0.0   
            2015-05-28             0.0         0.0         0.0         0.0   
...                                ...         ...         ...         ...   
310         2018-08-28             0.0         0.0         0.0         0.0   
            2018-09-06             0.0         0.0         0.0         0.0   
R1762       2015-02-24             0.0         0.0         0.0         0.0   
            2015-02-26             0.0         0.0         0.0         0.0   
R1764       2015-02-25             0.0         0.0         0.0         0.0   

                            SPN_523543  SPN_524033  SPN_524037  SPN_524071  \
EquipmentID EventTimeStamp                                                   
301         2015-05-11             0.0         0.0         0.0         0.0   
            2015-05-13             0.0         0.0         0.0         0.0   
            2015-05-18             0.0         0.0         0.0         0.0   
            2015-05-21             0.0         0.0         0.0         0.0   
            2015-05-28             0.0         0.0         0.0         0.0   
...                                ...         ...         ...         ...   
310         2018-08-28             0.0         0.0         0.0         0.0   
            2018-09-06             0.0         0.0         0.0         0.0   
R1762       2015-02-24             0.0         0.0         0.0         0.0   
            2015-02-26             0.0         0.0         0.0         0

## Model Training and Evaluation

### Scale and Train/Test split the Data

In [6]:
# Scale the data
scaler = StandardScaler()

# Train logistic regression model
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

ValueError: could not convert string to float: 'R1762'

### Train Logistic Regression Model

In [ ]:
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

### Evaluate the model

In [ ]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
probs = clf.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
plt.plot([0, 1], [0, 1], linestyle="--", label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend()
plt.show()

NameError: name 'confusion_matrix' is not defined